In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import torchvision
import torchvision.datasets as datasets

import numpy as np
from pathlib import Path
from PIL import Image

from utils import resize_and_crop

### Load the dataset

In [3]:
np.random.seed(0)

In [4]:
mnist_train = datasets.MNIST(root="./data", train=True, download=True, transform=None)
X_train, y_train = mnist_train.data, mnist_train.targets
mnist_test = datasets.MNIST(root="./data", train=False, download=True, transform=None)
X_test, y_test = mnist_test.data, mnist_test.targets

In [5]:
# Only the digits 0, 1, and 2 were used in the target dataset
target_idx = np.where(y_train < 3)[0] 
foreground = X_train[target_idx,:][:10000].reshape(-1, 28*28) / 256.
target_labels = y_train[target_idx][:10000]

In [14]:
IMAGENET_PATH = "./data/imagenet/grass_images/"

grass_imgs = []
for fpath in Path(IMAGENET_PATH).iterdir():
    if fpath.suffix != ".JPEG": 
        print("unknown file extension:", fpath)
        continue
    
    try:
        im = Image.open(fpath)
        im = im.convert(mode="L") # grayscale
        im = resize_and_crop(im) # resize and crop to 100px x 100px
        grass_imgs.append(np.reshape(im, 10000))
    except:
        print("unknown error:", fpath)
    
grass_imgs = np.asarray(grass_imgs, dtype=float) / 255. # rescale to 0-1
print(grass_imgs.shape)

(1274, 10000)


In [19]:
rand_idxs = np.random.permutation(grass_imgs.shape[0])
split = int(len(rand_idxs)) // 2

target_idxs = rand_idxs[:split]
background_idxs = rand_idxs[split:]

target = np.zeros(foreground.shape)
background = np.zeros(foreground.shape)

for i in range(target.shape[0]):
    idx = np.random.choice(target_idxs) # randomly pick an image
    